In [379]:
import numpy as np
import pandas as pd

In [380]:
df = pd.read_csv('sales_pos.csv')

In [381]:
df1 = df.copy()
df1.shape

(550068, 11)

In [382]:
df1.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


### pivot_table

In [383]:
# prod 별 각 도시의 purchase 평균, 표준편차

pv_df1 = pd.pivot_table(df1, index='prod',columns=['city'],values='purchase',aggfunc=['mean','std'])
pv_df1.head(2)

mean                                      std               \
city                  A             B             C            A            B   
prod                                                                            
P00000142  11077.560784  11174.903448  11150.681818  2283.546514  2280.121291   
P00000242   9945.843373  10377.618785  11282.517857  3681.578007  3028.912919   

                        
city                 C  
prod                    
P00000142  2344.426206  
P00000242  2865.131524

In [384]:
# B 도시에서 구매 평균이 가능 높은 상품은?
pv_df1['mean']['B'].idxmax()

'P00086242'

### multi index : xs()

In [385]:
# df['상위','하위'] 로 selecting  
# pv_df1.xs(('mean','B'),level=[0,1],axis=1)

pv_df1['mean','B'].idxmax()  

# multi index 가 아니면 오류가 난다.df['a','b']

'P00086242'

### stack

In [386]:
# stack : column 을 index 로 이동 
# pivot (columns)-> stack -> groupby (index) 이랑 동일하게 만들수 있네.

pv_df1.stack(level=1)     # level=0 은 mean,std 가 인덱스로 이동

mean          std
prod      city                           
P00000142 A     11077.560784  2283.546514
          B     11174.903448  2280.121291
          C     11150.681818  2344.426206
P00000242 A      9945.843373  3681.578007
          B     10377.618785  3028.912919
...                      ...          ...
P0099842  B      6838.640000  1870.601690
          C      7606.575758  1336.883223
P0099942  A      6686.600000  1463.875609
          B      4712.875000  2072.996826
          C      6883.000000          NaN

[10321 rows x 2 columns]

In [387]:
# 이게 group 이랑 같은 거네.
pd.pivot_table(df1, index=['prod','city'],values='purchase',aggfunc=['mean','std'])

mean          std
                    purchase     purchase
prod      city                           
P00000142 A     11077.560784  2283.546514
          B     11174.903448  2280.121291
          C     11150.681818  2344.426206
P00000242 A      9945.843373  3681.578007
          B     10377.618785  3028.912919
...                      ...          ...
P0099842  B      6838.640000  1870.601690
          C      7606.575758  1336.883223
P0099942  A      6686.600000  1463.875609
          B      4712.875000  2072.996826
          C      6883.000000          NaN

[10321 rows x 2 columns]

In [388]:
# A 도시의 평균, 표준편차 만 

pd.pivot_table(df1, index=['prod','city'],values='purchase',aggfunc=['mean','std']).xs('A',level=1)

,mean,std
,purchase,purchase
prod,,
P00000142,11077.560784,2283.546514
P00000242,9945.843373,3681.578007
P00000342,5205.015625,1816.543041
P00000442,4696.583333,1608.706053
P00000542,4620.159091,1590.366580
...,...,...
P0099442,13718.107692,3758.431493
P0099642,5538.000000,879.767772


### groupby.apply()

In [389]:
def funcX(groupKey):
    return groupKey.sort_values(by='purchase',ascending=False)[:3]

df1.groupby(df.city).apply(funcX)

# groupby 의 key 값 (그룹으로 묶여진 df의 key) 으로  A, B, C 가 전달된다. 


user       prod gender age_group  job city  marital  prod_cat1  \
city                                                                          
A    87440   1474  P00052842      M     26-35    4    A        1         10   
     349658  5848  P00119342      M     51-55   20    A        1         10   
     229329  5367  P00085342      M     18-25    4    A        0         10   
B    292083  3045  P00052842      M     46-50    1    B        1         10   
     503697  5596  P00117642      M     36-45   12    B        0         10   
     437804  1387  P00086242      F     51-55   13    B        1         10   
C    93016   2272  P00052842      M     26-35    0    C        0         10   
     370891  3160  P00052842      M     26-35   17    C        0         10   
     321782  1577  P00052842      M       55+    0    C        1         10   

             prod_cat2  prod_cat3  purchase  
city                                         
A    87440        15.0        NaN     23961  
     349658       13.0        NaN     23960  
     229329       13.0        NaN     23958  
B    292083       15.0        NaN     23960  
     503697       16.0        NaN     23960  
     437804        NaN        NaN     23959  
C    93016        15.0        NaN     23961  
     370891       15.0        NaN     23961  
     321782       15.0        NaN     23960

In [390]:
df1.groupby(df.city).groups

{'A': [0, 1, 2, 3, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17, 56, 57, 58, 59, 60, 61, 62, 63, 64, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 124, 125, 126, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 234, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 297, 298, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, ...], 'B': [6, 7, 8, 18, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 97, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 220, 221, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, ...], 'C': [4, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 47, 48, 49

In [391]:
# 똑 같이 groupby 로 만들어 보자

df1.groupby(by=['prod','city'])[['purchase']].agg(['sum','std'])

purchase             
                    sum          std
prod      city                      
P00000142 A     2824778  2283.546514
          B     4861083  2280.121291
          C     5151615  2344.426206
P00000242 A      825505  3681.578007
          B     1878349  3028.912919
...                 ...          ...
P0099842  B      341932  1870.601690
          C      251017  1336.883223
P0099942  A       33433  1463.875609
          B       37703  2072.996826
          C        6883          NaN

[10321 rows x 2 columns]

In [392]:
# 조건별 groupby : purchase 

# x < 500
sel_pur1 = df1['purchase']<500
# 500<= x < 1000
sel_pur2 = (500<=df1['purchase'])&(df1['purchase']<=1000)
# 10000 < x
sel_pur3 = 1000<df1['purchase']

# Female
sel_Fe = df1['gender']=='F'

# level                       0   ,      1 ,     3  , 4
cond_group = df1.groupby([sel_pur1,sel_pur2,sel_pur3,sel_Fe])['purchase'].agg('sum')  # ['sum'] 하면 df 출력
cond_group.unstack() # level 4 가 column 으로 이동

gender                           False       True 
purchase purchase purchase                        
False    False    True      3905025334  1184608058
         True     False        3859170     1383929
True     False    False         695596      240655

In [393]:
cond_group[True,False,False]

gender
False    695596
True     240655
Name: purchase, dtype: int64

### unstack

In [394]:
# groupby (index) -> unstack -> pivot_table (columns)
# pd.pivot_table(df1, index='prod',columns=['city'],values='purchase',aggfunc=['mean','std']) 와 같다.

df1.groupby(by=['prod','city'])[['purchase']].agg(['sum','std']).unstack()

purchase                                                  \
                 sum                                std                
city               A          B          C            A            B   
prod                                                                   
P00000142  2824778.0  4861083.0  5151615.0  2283.546514  2280.121291   
P00000242   825505.0  1878349.0  1263642.0  3681.578007  3028.912919   
P00000342   333121.0   526030.0   437324.0  1816.543041  1908.004581   
P00000442   169077.0   192561.0    79535.0  1608.706053  1786.514719   
P00000542   203287.0   400975.0   202950.0  1590.366580  1439.073191   
...              ...        ...        ...          ...          ...   
P0099442    891677.0  1256504.0   722202.0  3758.431493  3613.476528   
P0099642     22152.0    42113.0    19445.0   879.767772  2378.339035   
P0099742    270270.0   386634.0   335044.0  3162.009313  2960.552363   
P0099842    144363.0   341932.0   251017.0  1155.428755  1870.601690   
P0099942     33433.0    37703.0     6883.0  1463.875609  2072.996826   

                        
                        
city                 C  
prod                    
P00000142  2344.426206  
P00000242  2865.131524  
P00000342  1704.636686  
P00000442  1487.392790  
P00000542  2090.943177  
...                ...  
P0099442   3592.391870  
P0099642   2955.543492  
P0099742   2695.186257  
P0099842   1336.883223  
P0099942           NaN  

[3631 rows x 6 columns]

In [395]:
df1.groupby(by=['prod','city'])[['purchase']].agg(['sum','std']).stack()

purchase
prod      city                  
P00000142 A    sum  2.824778e+06
               std  2.283547e+03
          B    sum  4.861083e+06
               std  2.280121e+03
          C    sum  5.151615e+06
...                          ...
P0099942  A    sum  3.343300e+04
               std  1.463876e+03
          B    sum  3.770300e+04
               std  2.072997e+03
          C    sum  6.883000e+03

[20016 rows x 1 columns]

### condition selectiong

In [396]:
rule1 = df1['prod'].str[1:].astype('int')%4 != 0
df1.loc[rule1,:].groupby(by=['prod'])['purchase'].agg(['sum'])   # ('sum') 하면 Series 리턴

# prod 값이 4의 배수가 아닌 데이터중에서 (rule1) , prod별 purchase 의 합이  .idxmax() 가장 큰 prod ?

,sum
prod,
P00000142,12837476
P00000242,3967496
P00000342,1296475
P00000442,441173
P00000542,807212
...,...
P0099442,2870383
P0099642,83710
P0099742,991948


In [397]:
# A 도시의 평균, 표준편차 만 

rule2 = df1.loc[:,'city'] =='A'
df1.loc[rule2,:].groupby('prod')[['purchase']].agg(['mean','std'])

purchase             
                   mean          std
prod                                
P00000142  11077.560784  2283.546514
P00000242   9945.843373  3681.578007
P00000342   5205.015625  1816.543041
P00000442   4696.583333  1608.706053
P00000542   4620.159091  1590.366580
...                 ...          ...
P0099442   13718.107692  3758.431493
P0099642    5538.000000   879.767772
P0099742    7949.117647  3162.009313
P0099842    7598.052632  1155.428755
P0099942    6686.600000  1463.875609

[3477 rows x 2 columns]

### Q1

In [398]:
top_prod = df1.groupby(by=['prod'])['purchase'].sum().idxmax()
top_prod

'P00025442'

df1.groupby('prod')['job'].value_counts()[top_prod]

In [399]:

# prod 들 중에서 가장 purchase 가 큰 prod 을 가장 많이 구매한 구매자의 job

df1.loc[df1['prod']==top_prod, 'job'].value_counts().idxmax()

# 확인 해야할 값들이 리스트인 경우 isin 사용, str 이면 [str,'a']
# df1.loc[df1['prod'].isin([top_prod]), 'job'].value_counts().idxmax()

# groupby 와 같다.
# df1.groupby('prod')['job'].value_counts()[top_prod]
# prod 로 그룹화된 job 값들중 prod가 top_prod만 표시 

4

### Q2

In [400]:
df2 = df.copy()

In [401]:
df2[['prod_cat1','prod_cat2','prod_cat3']]=df2[['prod_cat1','prod_cat2','prod_cat3']].fillna(0).astype('int')
df2.dtypes

user          int64
prod         object
gender       object
age_group    object
job           int64
city         object
marital       int64
prod_cat1     int64
prod_cat2     int64
prod_cat3     int64
purchase      int64
dtype: object

In [402]:
df2['prod_cat1'] =df2['prod_cat1'].astype('str')
df2['prod_cat2'] =df2['prod_cat2'].astype('str')
df2['prod_cat3'] =df2['prod_cat3'].astype('str')
df2['prod_cat']=df2['prod_cat1']+'-'+df2['prod_cat2']+'-'+df2['prod_cat3']
df2.head()

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase,prod_cat
0,1,P00069042,F,0-17,10,A,0,3,0,0,8370,3-0-0
1,1,P00248942,F,0-17,10,A,0,1,6,14,15200,1-6-14
2,1,P00087842,F,0-17,10,A,0,12,0,0,1422,12-0-0
3,1,P00085442,F,0-17,10,A,0,12,14,0,1057,12-14-0
4,2,P00285442,M,55+,16,C,0,8,0,0,7969,8-0-0


In [403]:
df2.loc[df2['user']==1,'prod_cat'].nunique()

# user1 인 고객은 총 21개 카테고리의 물품을 구매 하였다. 
# nunique() : unique 한 갯수

21

groupby

In [404]:
G2635 = df2['age_group']=='26-35'                                      # 26-36 대상으로
df2.loc[G2635,:].groupby(['user','marital'])[['prod_cat']].nunique()   # 고객, 결혼여부에 따른 카테고리 개수 산출

,,prod_cat
user,marital,
3,0,18
5,1,43
8,1,32
9,0,31
11,0,34
...,...,...
6030,1,33
6034,0,8
6035,0,61


mean().unstack() 


In [405]:

A = df2.loc[G2635,:].groupby(['user','marital'])[['prod_cat']].nunique().unstack().mean()[0]
B = df2.loc[G2635,:].groupby(['user','marital'])[['prod_cat']].nunique().unstack().mean()[1]
abs(A-B)

0.12915293780976356

xs(index,level=level of index, axis=).mean()

In [406]:

A0 = df2.loc[G2635,:].groupby(['user','marital'])[['prod_cat']].nunique().xs(0,level=1).mean()
A1 = df2.loc[G2635,:].groupby(['user','marital'])[['prod_cat']].nunique().xs(1,level=1).mean()
abs(A0-A1)

prod_cat    0.129153
dtype: float64

Pivot table = groupby + unstack

In [407]:
mypi = pd.pivot_table(df2,index=['age_group','user'],columns=['marital'],values=['prod_cat'],aggfunc=('nunique'))
mypi

prod_cat      
marital               0     1
age_group user               
0-17      1        21.0   NaN
          19       59.0   NaN
          51       12.0   NaN
          75       36.0   NaN
          86       19.0   NaN
...                 ...   ...
55+       5948     67.0   NaN
          5968      NaN  14.0
          5980     66.0   NaN
          5986      NaN  19.0
          6038      9.0   NaN

[5891 rows x 2 columns]

In [408]:
mypi.query("age_group==['26-35']").mean()

          marital
prod_cat  0          41.663183
          1          41.792336
dtype: float64

In [409]:
mypi.xs('26-35',level=0,axis=0).mean()

          marital
prod_cat  0          41.663183
          1          41.792336
dtype: float64

In [410]:
mypi.loc['26-35',:].mean()

          marital
prod_cat  0          41.663183
          1          41.792336
dtype: float64

### Q3

In [411]:
df3=df.copy()

In [412]:
df3.tail()

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
550063,6033,P00372445,M,51-55,13,B,1,20,NaN,NaN,368
550064,6035,P00375436,F,26-35,1,C,0,20,NaN,NaN,371
550065,6036,P00375436,F,26-35,15,B,1,20,NaN,NaN,137
550066,6038,P00375436,F,55+,1,C,0,20,NaN,NaN,365
550067,6039,P00371644,F,46-50,0,B,1,20,NaN,NaN,490


In [413]:
df3.groupby('user')[['prod']].nunique()

,prod
user,
1,35
2,77
3,29
4,14
5,106
...,...
6036,514
6037,122
6038,12


In [414]:
df3[['user','gender','age_group','job','city','marital']].drop_duplicates()

,user,gender,age_group,job,city,marital
0,1,F,0-17,10,A,0
4,2,M,55+,16,C,0
5,3,M,26-35,15,A,0
6,4,M,46-50,7,B,1
9,5,M,26-35,20,A,1
...,...,...,...,...,...,...
185450,4588,F,26-35,4,C,0
187076,4871,M,18-25,12,C,0
221494,4113,M,36-45,17,C,0
229480,5391,M,26-35,7,A,0


pivot_table(df,index=[],values=[],aggfunc={})

In [415]:
df3_1 = pd.pivot_table(df3,index=['user','gender','age_group','job','city','marital'],values=['prod','purchase'],aggfunc={'prod':'nunique','purchase':'sum'}).reset_index()
df3_1.tail()

,user,gender,age_group,job,city,marital,prod,purchase
5886,6036,F,26-35,15,B,1,514,4116058
5887,6037,F,46-50,1,C,0,122,1119538
5888,6038,F,55+,1,C,0,12,90034
5889,6039,F,46-50,0,B,1,74,590319
5890,6040,M,26-35,6,B,0,180,1653299


In [416]:
df3_1['gender'] = np.where(df3_1['gender']=='M', 1, 0 )
# df3_1['gender'] = df3_1['gender'].replace({'M':1, 'F':0})
df3_1.tail()

,user,gender,age_group,job,city,marital,prod,purchase
5886,6036,0,26-35,15,B,1,514,4116058
5887,6037,0,46-50,1,C,0,122,1119538
5888,6038,0,55+,1,C,0,12,90034
5889,6039,0,46-50,0,B,1,74,590319
5890,6040,1,26-35,6,B,0,180,1653299


In [417]:
a_list =df3_1['age_group'].unique()
a_list

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [418]:
pd.crosstab(df3_1['age_group'],df3_1['age_group'].replace(a_list,[0,6,2,4,5,3,1]))

age_group,0,1,2,3,4,5,6
age_group,,,,,,,
0-17,218,0,0,0,0,0,0
18-25,0,1069,0,0,0,0,0
26-35,0,0,2053,0,0,0,0
36-45,0,0,0,1167,0,0,0
46-50,0,0,0,0,531,0,0
51-55,0,0,0,0,0,481,0
55+,0,0,0,0,0,0,372


replace(list,list)

In [419]:
df3_1['age_group']=df3_1['age_group'].replace(a_list,[0,6,2,4,5,3,1])


In [420]:
df3_1 = pd.get_dummies(df3_1,columns=['job','city'])
df3_1.tail()

,user,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
5886,6036,0,2,1,514,4116058,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
5887,6037,0,4,0,122,1119538,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
5888,6038,0,6,0,12,90034,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
5889,6039,0,4,1,74,590319,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
5890,6040,1,2,0,180,1653299,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [421]:
from sklearn.preprocessing import MinMaxScaler
myMMS = MinMaxScaler()

In [422]:
df3_1.loc[:,'gender':'city_C']= myMMS.fit_transform(df3_1.loc[:,'gender':'city_C'])
df3_1

,user,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,1,0,0.000000,0,0.028431,0.027398,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2,1,1.000000,0,0.069608,0.072810,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,1,0.333333,0,0.022549,0.028117,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,1,0.666667,1,0.007843,0.015232,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5,1,0.333333,1,0.098039,0.073813,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5886,6036,0,0.333333,1,0.498039,0.387921,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5887,6037,0,0.666667,0,0.113725,0.102272,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5888,6038,0,1.000000,0,0.005882,0.004133,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5889,6039,0,0.666667,1,0.066667,0.051823,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [423]:
from sklearn.cluster import KMeans

In [424]:
myKM = KMeans(n_clusters=7, random_state=123)
myKM.fit(df3_1.loc[:,'gender':'city_C'])

/Users/smjune/Works/ipynb/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=7, random_state=123)

In [425]:
from sklearn.metrics import silhouette_score

In [426]:
silhouette_score(df3_1.loc[:,'gender':'city_C'],myKM.labels_)

0.17879229751009443